Dataset organized kindly by user Mourad. https://www.kaggle.com/msheriey/104-flowers-garden-of-eden

# import

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import random

# global variables

In [2]:
# image size options: 192, 224, 311, 512
IMAGE_DIMENSION = 224
VECTOR_LEN = IMAGE_DIMENSION**2
NUM_CLASS = 104

TRAIN_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/train'
VAL_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/val'
#TEST_DIR = f'data/jpeg-{IMAGE_DIMENSION}x{IMAGE_DIMENSION}/test/test'

BATCH_SIZE = 64
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
#TEST_BATCH_SIZE = 32

# generate datasets

In [3]:
def generate_train_dataset():
    train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        TRAIN_DIR,
        target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION), 
        batch_size=TRAIN_BATCH_SIZE,
        shuffle=True 
    )
    
    return next(train_generator)
        
def generate_val_dataset():
    val_generator = ImageDataGenerator().flow_from_directory(
        VAL_DIR,
        target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
        batch_size=VAL_BATCH_SIZE,
        shuffle=True
    )
    
    return next(val_generator)
        
def generate_test_dataset():
    test_generator = ImageDataGenerator().flow_from_directory(
        TEST_DIR,
        target_size=(IMAGE_DIMENSION, IMAGE_DIMENSION),
        batch_size=TEST_BATCH_SIZE,
        shuffle=False
    )
    
    return next(test_generator)

In [4]:
train_img, train_lb = generate_train_dataset() # x_train, y_train
val_img, val_lb = generate_val_dataset() # x_test, y_test
#test_img, test_lb = generate_test_dataset()

Found 12753 images belonging to 104 classes.
Found 3712 images belonging to 104 classes.


# explore data

In [5]:
train_img.shape, train_lb.shape

((64, 224, 224, 3), (64, 104))

# preprocessing

In [6]:
train_img.shape, train_lb.shape

((64, 224, 224, 3), (64, 104))

In [7]:
print(np.shape(train_img))
print(np.shape(train_lb))
print(np.shape(val_img))
print(np.shape(val_lb))

(64, 224, 224, 3)
(64, 104)
(64, 224, 224, 3)
(64, 104)


In [8]:
train_img_unrow = train_img.reshape(TRAIN_BATCH_SIZE, -1).T
np.shape(train_img_unrow)

(150528, 64)

In [9]:
val_img_unrow = val_img.reshape(VAL_BATCH_SIZE, -1).T
np.shape(val_img_unrow)

(150528, 64)

# cnn

In [10]:
# initial_learning_rate = 0.1
# lr_schedule = ExponentialDecay(
#     initial_learning_rate, decay_steps=100000, decay_rate=0.01, staircase=True
# )

# optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)

In [11]:
# Build the model with multiple hidden layers
model_1 = models.Sequential()
model_1.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_1.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_1.add(layers.Flatten())        
# possibly more layers here

model_1.add(layers.Dense(NUM_CLASS)) # output layer
model_1.add(layers.Activation('sigmoid'))

In [12]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 32)      0         
_________________________________________________________________
flatten (Flatten)            (None, 401408)            0         
_________________________________________________________________
dense (Dense)                (None, 104)               41746536  
_________________________________________________________________
activation (Activation)      (None, 104)               0         
Total params: 41,746,952
Trainable params: 41,746,952
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Define an optimizer, cost lost function, and scoring metric
model_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [14]:
model_1.fit(train_img, train_lb, epochs=50, batch_size=TRAIN_BATCH_SIZE)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-12-01 10:54:40.541135: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-12-01 10:54:40.541276: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 746ms/step - loss: 0.6972 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 398ms/step - loss: 0.1857 - accuracy: 0.0625
Epoch 3/50
1/1 [==============================] - 0s 383ms/step - loss: 0.2788 - accuracy: 0.1719
Epoch 4/50
1/1 [==============================] - 0s 381ms/step - loss: 0.3272 - accuracy: 0.1250
Epoch 5/50
1/1 [==============================] - 0s 393ms/step - loss: 0.3488 - accuracy: 0.1250
Epoch 6/50
1/1 [==============================] - 0s 378ms/step - loss: 0.3525 - accuracy: 0.1250
Epoch 7/50
1/1 [==============================] - 0s 389ms/step - loss: 0.3439 - accuracy: 0.1250
Epoch 8/50
1/1 [==============================] - 0s 377ms/step - loss: 0.3267 - accuracy: 0.1250
Epoch 9/50
1/1 [==============================] - 0s 395ms/step - loss: 0.3040 - accuracy: 0.1250
Epoch 10/50
1/1 [==============================] - 0s 383ms/step - loss: 0.2887 - accuracy: 0.1250
Epoch 11/50
1/1 [=========

In [15]:
val_loss, val_acc = model_1.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 185ms/step - loss: 66.0861 - accuracy: 0.0781


### Probably some overfitting in this cnn model. Will try adding more layers to this and see if that reduces overfitting

In [ ]:
# Build the model with multiple hidden layers
model_2 = models.Sequential()
model_2.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_2.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_2.add(layers.Flatten())        
# possibly more layers here
model_2.add(layers.Dense(128))
model_2.add(layers.Activation('relu'))
model_2.add(layers.Dense(NUM_CLASS)) # output layer
model_2.add(layers.Activation('sigmoid'))

In [ ]:
model_2.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_2.fit(train_img, train_lb, epochs=50, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_2.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### increase batch size and epochs to experiment

In [ ]:
BATCH_SIZE = 128
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
#TEST_BATCH_SIZE = 32

In [ ]:
# Build the model with multiple hidden layers
model_3 = models.Sequential()
model_3.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_3.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_3.add(layers.Flatten())        
# possibly more layers here
model_3.add(layers.Dense(128))
model_3.add(layers.Activation('relu'))
model_3.add(layers.Dense(NUM_CLASS)) # output layer
model_3.add(layers.Activation('sigmoid'))

In [ ]:
model_3.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_3.fit(train_img, train_lb, epochs=100, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_3.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### adding dropout layer

In [ ]:
BATCH_SIZE = 128
TRAIN_BATCH_SIZE = BATCH_SIZE
VAL_BATCH_SIZE = BATCH_SIZE
#TEST_BATCH_SIZE = 32

In [ ]:
# Build the model with multiple hidden layers
model_4 = models.Sequential()
model_4.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_4.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_4.add(layers.Flatten())        
# possibly more layers here
model_4.add(layers.Dropout(0.2, input_shape=(IMAGE_DIMENSION,)))
model_4.add(layers.Dense(128, activation='relu'))
model_4.add(layers.Dense(64, activation='relu'))
model_4.add(layers.Dense(NUM_CLASS)) # output layer
model_4.add(layers.Activation('sigmoid'))

In [ ]:
model_4.summary()

In [ ]:
# Define an optimizer, cost lost function, and scoring metric
model_4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_4.fit(train_img, train_lb, epochs=100, batch_size=TRAIN_BATCH_SIZE)

In [ ]:
val_loss, val_acc = model_4.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

### shuffle is turned on. more epochs

In [32]:
# Build the model with multiple hidden layers
model_1 = models.Sequential()
model_1.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(IMAGE_DIMENSION, IMAGE_DIMENSION, 3),
                        data_format = 'channels_last'))
model_1.add(layers.BatchNormalization())
model_1.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_1.add(layers.Flatten())        
# possibly more layers here
model_1.add(layers.Dense(NUM_CLASS)) # output layer
model_1.add(layers.Activation('sigmoid'))

In [33]:
model_1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 224, 224, 32)      416       
_________________________________________________________________
batch_normalization_2 (Batch (None, 224, 224, 32)      128       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 401408)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 104)               41746536  
_________________________________________________________________
activation_4 (Activation)    (None, 104)               0         
Total params: 41,747,080
Trainable params: 41,747,016
Non-trainable params: 64
_________________________________________

In [34]:
# Define an optimizer, cost lost function, and scoring metric
model_1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_crossentropy'])

In [35]:
model_1.fit(train_img, train_lb, epochs=150, batch_size=TRAIN_BATCH_SIZE)

Epoch 1/150
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


ValueError: Shape mismatch: The shape of labels (received (6656,)) should equal the shape of logits except for the last dimension (received (64, 104)).

In [30]:
val_loss, val_acc = model_1.evaluate(val_img, val_lb, batch_size=TRAIN_BATCH_SIZE)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 254ms/step - loss: 157.7960 - accuracy: 0.0469


# data augmentation

running into overfitting issues. will augment data to reduce overfitting

In [ ]:
def random_fliplr(image):
    if random.choice([0,1]):
        image = np.fliplr(image)
    else:
        pass

    return image

def random_flipud(image):
    if random_choice([0,1]):
        image = np.flipud(image)
    else:
        pass

    return image

In [ ]:
# display a sample image
array_to_img(train_img[0])

In [ ]:
def augment_dataset(dataset):
    # actual augmenting the dataset
    augmented_dataset = dataset[0]
    print(augmented_dataset)
    
    for img in dataset:
        img = random_fliplr(img)
        img = random_flipud(img)
        
        np.append(augmented_dataset, img, axis=0)
        
    return augmented_dataset

In [ ]:
array_to_img(random_fliplr(train_img[0]))

In [ ]:
new_array = augment_dataset(train_img)

In [ ]:
def print_img(dataset):
    for img in dataset:
        array_to_img(img)
        
    return array_to_img(img)

In [ ]:
print_img(train_img)